In [2]:
df = df.sort_values(by="question_id")  # Or any other stable column

In [ ]:
!pip install transformers accelerate torch


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
pip install tiktoken==0.4.0


In [10]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/local/bin/huggingface-c

In [13]:
import torch
import pandas as pd
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load Meta Llama 3 Model and Tokenizer
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Load the inefficient code JSON file
with open("/kaggle/input/newdata/balanced_samples.json", "r") as f:
    inefficient_data = json.load(f)

# Function to optimize inefficient code using Llama-3
def optimize_code(inefficient_code):
    # Construct prompt for Llama-3 without reasoning
    prompt = f"### Task: Optimize the following Python code to improve efficiency and make it more concise. Do not explain or use comments, only return the optimized code. Give a code according to Python 3.8 and the whole answer should be enclosed in a Class Solution and function name should be the same as submitted to you in the input code\n\n#### Input Code:\n{inefficient_code}\n\n#### Optimized Code:\n"

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    try:
        # Generate optimized code
        outputs = model.generate(**inputs, max_new_tokens=500, temperature=0.7, top_p=0.9)

        # Decode generated text
        optimized_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract the part after "#### Optimized Code:"
        return optimized_code.split("#### Optimized Code:")[-1].strip()

    except Exception as e:
        print(f"Error optimizing code: {e}")
        return None

# Process inefficient code from JSON file
optimized_results = []
for entry in inefficient_data:
    try:
        # Extract question_id and inefficient code
        question_id = entry["question_id"]
        inefficient_code = entry["runtime_inefficient_codes"][0]["code"]

        # Optimize with Llama-3
        optimized_code = optimize_code(inefficient_code)

        if optimized_code:
            optimized_results.append({
                "question_id": question_id,
                "optimized_code": optimized_code
            })
        else:
            print(f"Skipping optimization for question_id {question_id} due to error.")

    except Exception as e:
        print(f"Error processing question_id {entry['question_id']}: {e}")

# Convert results to DataFrame and save to JSON
optimized_df = pd.DataFrame(optimized_results)

# Define the output path in Kaggle's working directory
output_path = "/kaggle/working/Meta_vanilla_prompting.json"

# Save the DataFrame as a JSON file with just the required columns
optimized_df.to_json(output_path, orient="records", indent=4)

# Print the output path to verify where the file is saved
print(f"File saved at {output_path}")

print(f"✅ Optimization complete! Results saved to {output_path}")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

File saved at /kaggle/working/optimized_test_results_meta.json
✅ Optimization complete for first 200 problems! Results saved to /kaggle/working/optimized_test_results_meta.json
